In [182]:
# import modules
# add path
import sys, os
sys.path.append(os.path.abspath("/Users/karl/Documents/Git/A2C_test"))
# modules used in training
import copy
import glob
import os
import time
from collections import deque
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import algo
from arguments import get_args
from envs import make_vec_envs
from model import Policy
from storage import RolloutStorage
from utils import get_vec_normalize, update_mode, neuro_activity, obs_representation
from tensorboardX import SummaryWriter
from distributions import Categorical, DiagGaussian
from utils import init, init_normc_, tanh_g
FixedCategorical = torch.distributions.Categorical
from matplotlib_venn import venn3
import matplotlib.pyplot as plt
import cv2
cv2.ocl.setUseOpenCL(False)
from PIL import Image

In [184]:
model_path = "/Users/karl/Documents/Git/A2C_test/Result1.0/Nomodulation/tanh_nomodulation_norm/Jan11_19-46-44_jinwei-a2c-10/MsPacmanNoFrameskip-v0.pt"
actor_critic = torch.load(model_path)

/anaconda3/lib/python3.6/site-packages/torch/serialization.py:425: SourceChangeWarning: source code of class 'model.Policy' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [185]:
dist = actor_critic.dist

In [186]:
features = torch.rand(32,512)

In [187]:
dist = actor_critic.dist(features)

In [189]:
dist.entropy().mean()

tensor(0.3036, grad_fn=<MeanBackward1>)

In [75]:
np.float32(obs).dtype

dtype('float32')

In [158]:
def reform(obs):
    frame = cv2.cvtColor(obs, cv2.COLOR_RGB2GRAY)
    frame = frame[:172,:]
    return frame

def make_input(last_frames, obs):
    all_frames = []
    for frame in last_frames:
        all_frames.append(frame)
    all_frames.append(obs)
    all_frames = np.stack(all_frames, axis=0)
    all_frames = np.float32(np.expand_dims(all_frames, axis=0))
    return torch.from_numpy(all_frames)

def smooth(scalars, weight):  # Weight between 0 and 1
    last = scalars[0]  # First value in the plot (first timestep)
    smoothed = list()
    for point in scalars:
        smoothed_val = last * weight + (1 - weight) * point  # Calculate smoothed value
        smoothed.append(smoothed_val)                        # Save it
        last = smoothed_val                                  # Anchor the last smoothed value

    return smoothed

In [80]:
#init
env = gym.make("MsPacmanNoFrameskip-v0")
obs = env.reset()
obs = reform(obs)/255
last_frames = deque(maxlen=3)
for i in range(3):
    last_frames.append(obs)

In [87]:
#run
obs_hist = []
g = torch.ones(1,1)
rnn_hxs = torch.zeros(1,1)
masks = torch.zeros(1,1)
step = 0
obs = env.reset()
obs_hist.append(obs)
obs = reform(obs)/255
values = []
rewards = []
while(True):
    step += 1
    if step%100 == 0:
        print("steps:", step)
    inputs = make_input(last_frames, obs)
    last_frames.append(obs)

    value, actor_features, rnn_hxs, x = actor_critic.base(inputs, g, rnn_hxs, masks)
    values.append(value.item())
    dist = actor_critic.dist(actor_features)
    action = dist.mode().item()
    obs, reward, done, infos = env.step(action)
    rewards.append(reward)
    if done:
        print("rewards:", sum(rewards))
        break
    obs_hist.append(obs)
    obs = reform(obs)/255
    

steps: 100
steps: 200
steps: 300
steps: 400
steps: 500
steps: 600
steps: 700
steps: 800
steps: 900
steps: 1000
steps: 1100
steps: 1200
steps: 1300
steps: 1400
steps: 1500
steps: 1600
steps: 1700
steps: 1800
steps: 1900
steps: 2000
steps: 2100
steps: 2200
steps: 2300
steps: 2400
steps: 2500
steps: 2600
steps: 2700
steps: 2800
steps: 2900
steps: 3000
steps: 3100
steps: 3200
steps: 3300
steps: 3400
steps: 3500
steps: 3600
steps: 3700
steps: 3800
steps: 3900
steps: 4000
steps: 4100
steps: 4200
steps: 4300
rewards: 3190.0


In [89]:
len(obs_hist)

4377

In [92]:
pderrors = []
for i in range(len(obs_hist)-1):
    pderrors.append(rewards[i]+values[i+1]-values[i])

In [160]:
smooth_errors = smooth(pderrors, 0.75)

In [95]:
pderrors = [0] + pderrors

In [162]:
error_dic = {}
for i in range(len(smooth_errors)):
    error_dic[i] = smooth_errors[i]

In [163]:
d = sorted(error_dic, key=error_dic.__getitem__)

In [117]:
def show_fig(k, error_dic, obs_hist):
    print("prediction error: ", error_dic[k])
    Image.fromarray(obs_hist[k], 'RGB').show()

In [181]:
show_fig(1092, error_dic, obs_hist)

prediction error:  -5.987697421508977


In [164]:
d[:15]

[722, 957, 585, 724, 718, 1080, 1094, 716, 437, 477, 706, 462, 1092, 1099, 720]

In [174]:
d[-50:-20]

[1541,
 541,
 540,
 649,
 2308,
 1914,
 735,
 1913,
 1936,
 1308,
 608,
 1307,
 2115,
 1540,
 2307,
 648,
 607,
 734,
 1935,
 647,
 1539,
 733,
 2306,
 1934,
 606,
 646,
 1538,
 2305,
 732,
 1933]

In [153]:
values[438]

110.0562515258789

In [154]:
values[437]

62.513336181640625